In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import os.path
import json 
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification



Bad key savefig.frameon in file C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probably 

In [2]:
# create folder inside the current directory to hold the processed data
try:
    Path('data').mkdir(parents=True, exist_ok=False)
    print ('creating data directory')
except:
    print ('directory already exists')
    pass

directory already exists


In [3]:
# path to each file in the all_data folder
all_paths = glob.glob('raw_data/*.csv')

# list to append df's to
list_of_df = []

# loop through all paths and append each csv as a df
for filename in all_paths:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_of_df.append(df)

# concat all df's into one df
master_df = pd.concat(list_of_df, axis=0, ignore_index=True)

# save df as master csv
master_df.to_csv('data/master_df.csv', index=False)


In [4]:
def make_json(string):
    '''
    converts the string representation of a json object into a python dict.
    returns np.nan if the string contains commas.
    '''
    try:
        # replace single quotations to make the string represent a JSON object
        json_acceptable_string = string.replace("'", "\"")
        return(json.loads(json_acceptable_string))
    except:
        # if the string is still not JSON compatible return np.nan
        return np.nan

In [5]:
def unpack(data_frame, column):
    '''
    unpacks dict in column to dataframe.
    '''
    # convert all strings into JSON objects
    unpacked = data_frame[column].apply(make_json).to_frame()

    # unpack JSON into DataFrame
    return pd.json_normalize(unpacked[column])

In [6]:
# define which columns are represented as dicts
unpack_list = ['category', 'creator', 'location', 'photo', 'profile', 'urls']

# init empty df 
unpacked_df = pd.DataFrame()

# unpack each column of dicts, save each as their own csv
for value in unpack_list:
    frame = unpack(master_df, value)
    frame.to_csv(f'data/{value}.csv', index=False)  

In [7]:
# read category data for name and parent name
df_category = pd.read_csv('data/category.csv')
df_category = df_category[['name', 'parent_name']]

In [8]:
# read category data for name and parent name
df_category = pd.read_csv('data/category.csv')
df_category = df_category[['name', 'parent_name']]
nlp_data = df[['blurb', 'slug', 'country']] 

In [9]:
# drop columns deemed unessecary
model_data = master_df.drop(columns = ['backers_count','usd_pledged','is_starrable','current_currency','static_usd_rate','usd_exchange_rate','usd_type','id','name', 'slug', 'category', 'creator', 'location', 'photo', 'profile', 'urls', 'country_displayable_name', 'currency_symbol', 'currency_trailing_code', 'disable_communication', 'source_url', 'currency', 'pledged', 'blurb'])

# create goal_usd column so that all goal amounts are in the same units, drop fx_rate and goals afterward
model_data['goal_usd'] = model_data['fx_rate']*model_data['goal']
model_data = model_data.drop(columns=['fx_rate', 'goal'])

# combine category data and all other date
model_data = pd.concat([model_data, df_category], axis=1, join='inner')

# create percentage funded column as another metric
model_data['percentage_funded'] = model_data.converted_pledged_amount/model_data.goal_usd*100

# create total days active column as another metric
model_data['total_days_active'] = (model_data.deadline-model_data.created_at)*0.00001157
model_data['launch_time'] = (model_data.launched_at-model_data.created_at)*0.00001157
model_data = model_data.drop(columns=['state_changed_at', 'deadline', 'launched_at'])
model_data

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time
0,5034,US,1609376406,True,False,successful,5000.000000,Cookbooks,Food,100.680000,29.994357,1.957378
1,48365,HK,1606278560,True,True,successful,44624.926500,Cookbooks,Food,108.381131,90.449007,30.470127
2,98,ES,1606477096,True,False,successful,54.095050,Cookbooks,Food,181.162603,16.466933,3.172101
3,127765,US,1604500905,True,True,successful,65000.000000,Cookbooks,Food,196.561538,64.574056,19.589896
4,14574,GB,1605454727,True,True,successful,13823.881138,Cookbooks,Food,105.426254,36.088774,6.099334
...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,US,1434905504,True,False,successful,3000.000000,Fiction,Publishing,113.166667,104.846831,74.857391
30988,21112,US,1440643700,True,False,successful,12000.000000,Fiction,Publishing,175.933333,38.111337,8.121897
30989,2000,US,1441111356,True,False,successful,2000.000000,Fiction,Publishing,100.000000,60.256653,0.277773
30990,3125,US,1436479764,True,False,successful,3000.000000,Fiction,Publishing,104.166667,83.577804,53.588364


In [10]:
# init encoder for x values
encoder = LabelEncoder()

# encode boolean and string type columns
encoding_columns = (model_data.select_dtypes(include=['bool', 'object'])).columns
for column in encoding_columns:
    model_data[column] = encoder.fit_transform(model_data[column])

In [11]:
model_data

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time
0,5034,24,1609376406,1,0,3,5000.000000,7,4,100.680000,29.994357,1.957378
1,48365,11,1606278560,1,1,3,44624.926500,7,4,108.381131,90.449007,30.470127
2,98,7,1606477096,1,0,3,54.095050,7,4,181.162603,16.466933,3.172101
3,127765,24,1604500905,1,1,3,65000.000000,7,4,196.561538,64.574056,19.589896
4,14574,9,1605454727,1,1,3,13823.881138,7,4,105.426254,36.088774,6.099334
...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,24,1434905504,1,0,3,3000.000000,13,7,113.166667,104.846831,74.857391
30988,21112,24,1440643700,1,0,3,12000.000000,13,7,175.933333,38.111337,8.121897
30989,2000,24,1441111356,1,0,3,2000.000000,13,7,100.000000,60.256653,0.277773
30990,3125,24,1436479764,1,0,3,3000.000000,13,7,104.166667,83.577804,53.588364


In [12]:
# Convert 'created at' to datetime format
import datetime
model_data['Date Created'] = pd.to_datetime(model_data['created_at'], unit='s')
model_data['Date Created'] = model_data['Date Created'].dt.strftime('%Y/%m/%d')
model_data

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time,Date Created
0,5034,24,1609376406,1,0,3,5000.000000,7,4,100.680000,29.994357,1.957378,2020/12/31
1,48365,11,1606278560,1,1,3,44624.926500,7,4,108.381131,90.449007,30.470127,2020/11/25
2,98,7,1606477096,1,0,3,54.095050,7,4,181.162603,16.466933,3.172101,2020/11/27
3,127765,24,1604500905,1,1,3,65000.000000,7,4,196.561538,64.574056,19.589896,2020/11/04
4,14574,9,1605454727,1,1,3,13823.881138,7,4,105.426254,36.088774,6.099334,2020/11/15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,24,1434905504,1,0,3,3000.000000,13,7,113.166667,104.846831,74.857391,2015/06/21
30988,21112,24,1440643700,1,0,3,12000.000000,13,7,175.933333,38.111337,8.121897,2015/08/27
30989,2000,24,1441111356,1,0,3,2000.000000,13,7,100.000000,60.256653,0.277773,2015/09/01
30990,3125,24,1436479764,1,0,3,3000.000000,13,7,104.166667,83.577804,53.588364,2015/07/09


In [13]:
# Drop NA's
model_data = model_data.dropna()
model_data

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time,Date Created
0,5034,24,1609376406,1,0,3,5000.000000,7,4,100.680000,29.994357,1.957378,2020/12/31
1,48365,11,1606278560,1,1,3,44624.926500,7,4,108.381131,90.449007,30.470127,2020/11/25
2,98,7,1606477096,1,0,3,54.095050,7,4,181.162603,16.466933,3.172101,2020/11/27
3,127765,24,1604500905,1,1,3,65000.000000,7,4,196.561538,64.574056,19.589896,2020/11/04
4,14574,9,1605454727,1,1,3,13823.881138,7,4,105.426254,36.088774,6.099334,2020/11/15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,24,1434905504,1,0,3,3000.000000,13,7,113.166667,104.846831,74.857391,2015/06/21
30988,21112,24,1440643700,1,0,3,12000.000000,13,7,175.933333,38.111337,8.121897,2015/08/27
30989,2000,24,1441111356,1,0,3,2000.000000,13,7,100.000000,60.256653,0.277773,2015/09/01
30990,3125,24,1436479764,1,0,3,3000.000000,13,7,104.166667,83.577804,53.588364,2015/07/09


In [14]:
# Create validation dataset
val_start_date = '2021/02/01'
val_end_date = '2021/08/07'
val_mask = (model_data['Date Created'] >= val_start_date) & (model_data['Date Created'] <= val_end_date)
val_dataset = model_data[val_mask]
val_dataset

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time,Date Created
96,2696,1,1625314596,1,0,3,407.706392,28,2,661.260174,30.477393,16.482321,2021/07/03
97,2949,14,1624865518,1,0,3,194.773500,28,2,1514.066339,81.117848,21.138968,2021/06/28
98,84407,3,1625252511,1,0,3,9930.636500,28,2,849.965659,51.606377,16.618697,2021/07/02
99,222818,24,1625077438,1,1,3,50000.000000,28,2,445.636000,48.520692,18.531252,2021/06/30
100,9715,3,1625034114,1,1,3,8897.850304,28,2,109.183675,61.629757,18.673193,2021/06/30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29726,1200,24,1618924995,0,0,1,5000.000000,27,5,24.000000,60.404992,0.426112,2021/04/20
29728,2,5,1618215205,0,0,1,5409.504950,19,10,0.036972,30.262295,0.272855,2021/04/12
29729,0,16,1617895322,0,0,1,19857.958785,40,5,0.000000,60.485623,1.036279,2021/04/08
29730,298,15,1616497370,0,0,1,5950.455445,41,5,5.008020,73.339048,13.235212,2021/03/23


In [15]:
# drop rows that have state 'live' or 'cancelled'
#model_data = model_data.drop(model_data[model_data['state'] == 'live'].index, axis=0)
#odel_data = model_data.drop(model_data[model_data['state'] == 'canceled'].index, axis=0)
#model_data = model_data.reset_index(drop=True)
#model_data.head()
#model_data.head()

In [16]:
# split into X and y variables
X = val_dataset.drop(columns=['state','Date Created'])
y = val_dataset.state

In [17]:
# generate 70% train/validation split 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,
                                                     random_state = 1,
                                                     stratify = y)

In [18]:
print("Build Neural Networks classifier")

Build Neural Networks classifier


In [129]:
# init and fit Neural Networks classifier
X = [[0., 0.], [1., 1.]]
y = [0, 1]
nn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                       hidden_layer_sizes=(4, 2), random_state=1)

nn_clf.fit(X, y)
#MLPClassifier(alpha=1e-05, hidden_layer_sizes=(6, 2), random_state=1,
              #solver='lbfgs')
#nn_clf.fit(X_train_scaled, y_train.state)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 2), random_state=1,
              solver='lbfgs')

In [130]:
# Fit the data
nn_clf.fit(X_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 2), random_state=1,
              solver='lbfgs')

In [131]:
# Score the accuracy
print(f"Training Data Score: {nn_clf.score(X_train, y_train)}")
print(f"Testing Data Score: {nn_clf.score(X_val, y_val)}")

Training Data Score: 0.6199034201224672
Testing Data Score: 0.8374455732946299


In [132]:
# Make predictions using the test data
predictions = nn_clf.predict(X_val)
results = pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_val
}).reset_index(drop=True)
results.tail()

,Prediction,Actual
684,3,3
685,3,3
686,3,1
687,3,3
688,3,3


In [133]:
# Display the confusion matrix

cm = confusion_matrix(y_val, predictions)
cm_df = pd.DataFrame(
cm, index=['Actual 0', 'Actual 1', 'Actual 2', 'Actual 3'], columns=['Predicted 0', 'Predicted 1', 'Predicted 2', 'Predicted 3']
)
# Displaying results
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,0,0,0,11
Actual 1,0,0,0,92
Actual 2,0,0,0,9
Actual 3,0,0,0,577


In [134]:
# init standard scaler
scaler = MinMaxScaler()

# scale Train and testing set, columns=X.columns to keep column headers.
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train))
X_test_scaled = pd.DataFrame(scaler.transform(X_test))
X_train_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000e+00,1.000000,0.503410,0.0,0.0,2.499932e-04,0.255814,0.4,0.000000e+00,0.008701,0.000005
1,7.184914e-07,0.375000,0.476156,0.0,0.0,6.520631e-04,0.790698,0.8,6.690614e-09,0.076686,0.068599
2,5.987428e-07,1.000000,0.624696,0.0,0.0,1.493246e-06,0.534884,0.5,2.423749e-06,0.058515,0.054795
3,1.532782e-06,0.208333,0.853909,1.0,0.0,2.096262e-07,0.302326,0.7,4.301316e-05,0.012599,0.000910
4,2.059675e-05,1.000000,0.427250,0.0,0.0,2.999325e-05,0.093023,0.3,4.168848e-06,0.145933,0.138441


In [135]:
# Calculate classification report
#from sklearn.metrics import classification_report
print(classification_report(y_val, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00        92
           2       0.00      0.00      0.00         9
           3       0.84      1.00      0.91       577

    accuracy                           0.84       689
   macro avg       0.21      0.25      0.23       689
weighted avg       0.70      0.84      0.76       689



C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [136]:
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
scaler = preprocessing.StandardScaler().fit(X_train)
scaler
StandardScaler()

scaler.mean_
#array([1. ..., 0. ..., 0.33...])

scaler.scale_
#array([0.81..., 0.81..., 1.24...])

X_scaled = scaler.transform(X_train)
X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [137]:
# Create train/test dataset
train_test_dataset = model_data.loc[val_mask == False]
train_test_dataset = train_test_dataset.drop(['Date Created'], axis=1)
train_test_dataset

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time
0,5034,24,1609376406,1,0,3,5000.000000,7,4,100.680000,29.994357,1.957378
1,48365,11,1606278560,1,1,3,44624.926500,7,4,108.381131,90.449007,30.470127
2,98,7,1606477096,1,0,3,54.095050,7,4,181.162603,16.466933,3.172101
3,127765,24,1604500905,1,1,3,65000.000000,7,4,196.561538,64.574056,19.589896
4,14574,9,1605454727,1,1,3,13823.881138,7,4,105.426254,36.088774,6.099334
...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,24,1434905504,1,0,3,3000.000000,13,7,113.166667,104.846831,74.857391
30988,21112,24,1440643700,1,0,3,12000.000000,13,7,175.933333,38.111337,8.121897
30989,2000,24,1441111356,1,0,3,2000.000000,13,7,100.000000,60.256653,0.277773
30990,3125,24,1436479764,1,0,3,3000.000000,13,7,104.166667,83.577804,53.588364


In [138]:
# split into X and y variables
X = train_test_dataset.drop(columns=['state'])
y = train_test_dataset.state#.to_frame()


In [139]:
# generate 70% train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                       random_state = 1,
                                                       stratify = y)

In [140]:
# Fit the data
nn_clf.fit(X_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 2), random_state=1,
              solver='lbfgs')

In [141]:
# init standard scaler
scaler = StandardScaler()

# scale Train and testing set, columns=X.columns to keep column headers.
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
X_train_scaled.head()

,converted_pledged_amount,country,created_at,spotlight,staff_pick,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time
0,-0.046637,0.622086,-0.612076,-1.277071,-0.450349,-0.002407,-0.903791,-0.455475,-0.014432,-0.355266,-0.334502
1,-0.046540,-1.325784,-0.741280,-1.277071,-0.450349,0.051478,1.026783,1.017933,-0.014431,1.054969,1.081655
2,-0.046556,0.622086,-0.037092,-1.277071,-0.450349,-0.035710,0.103465,-0.087123,-0.014094,0.678030,0.796673
3,-0.046430,-1.845216,1.049544,0.783042,-0.450349,-0.035882,-0.735915,0.649581,-0.008436,-0.274410,-0.315809
4,-0.043847,0.622086,-0.973126,-1.277071,-0.450349,-0.031891,-1.491356,-0.823827,-0.013851,2.491364,2.523556


In [142]:
# Score the accuracy
print(f"Training Data Score: {nn_clf.score(X_train, y_train)}")
print(f"Testing Data Score: {nn_clf.score(X_test, y_test)}")

Training Data Score: 0.6199034201224672
Testing Data Score: 0.6198606271777003


In [143]:
X = [[0., 0.], [1., 1.]]
y = [0, 1]
nn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                       hidden_layer_sizes=(4, 2), random_state=1)



In [144]:
# Fit the data
nn_clf.fit(X_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 2), random_state=1,
              solver='lbfgs')

In [145]:
# Make predictions using the test data
predictions = nn_clf.predict(X_test)
results = pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_test
}).reset_index(drop=True)
results.tail()

,Prediction,Actual
8605,3,3
8606,3,3
8607,3,2
8608,3,3
8609,3,3


In [146]:
# print classification report
print(classification_report(y_test, prediction_nn))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       276
           1       0.34      1.00      0.51      2769
           2       0.00      0.00      0.00       228
           3       0.00      0.00      0.00      5337

    accuracy                           0.32      8610
   macro avg       0.08      0.25      0.13      8610
weighted avg       0.11      0.32      0.16      8610



C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [147]:
# view confusion matrix
cm = confusion_matrix(y_test, prediction_nn)
cm

array([[   0,  276,    0,    0],
       [   9, 2760,    0,    0],
       [   0,  228,    0,    0],
       [ 468, 4869,    0,    0]], dtype=int64)

In [148]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       276
           1       0.00      0.00      0.00      2769
           2       0.00      0.00      0.00       228
           3       0.62      1.00      0.77      5337

    accuracy                           0.62      8610
   macro avg       0.15      0.25      0.19      8610
weighted avg       0.38      0.62      0.47      8610



C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jimp\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [149]:
print("this doesnt form a very strong classifier for the purposes of predictability for success in Kickstarter funding projects")

this doesnt form a very strong classifier for the purposes of predictability for success in Kickstarter funding projects
